In [1]:
import pandas as pd;import math;import numpy as np; import geopandas as gpd; import matplotlib as plt; import gc; import sklearn; import re; import prophet

C:\Users\Asimina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load data

In [2]:
df1 = pd.read_csv("Data/siss_geo_indicatori_lineari_2016.csv", sep=",", encoding='utf-8')
df2 = pd.read_csv("Data/siss_geo_indicatori_lineari_2017.csv", sep=",", encoding='utf-8')
df3 = pd.read_csv("Data/siss_geo_indicatori_lineari_2018.csv", sep=",", encoding='utf-8')
df4 = pd.read_csv("Data/siss_geo_indicatori_lineari_2019.csv", sep=",", encoding='utf-8')
df5 = pd.read_csv("Data/siss_geo_indicatori_lineari_2020.csv", sep=",", encoding='utf-8')
df6 = pd.read_csv("Data/siss_geo_indicatori_lineari_2021.csv", sep=",", encoding='utf-8')
df7 = pd.read_csv("Data/siss_geo_indicatori_lineari_2022.csv", sep=",", encoding='utf-8')
df1['Year'] = 2016
df2['Year'] = 2017
df3['Year'] = 2018
df4['Year'] = 2019
df5['Year'] = 2020
df6['Year'] = 2021
df7['Year'] = 2022
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], sort=False)

C:\Users\Asimina\AppData\Local\Temp\ipykernel_25816\2450062250.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("Data/siss_geo_indicatori_lineari_2016.csv", sep=",", encoding='utf-8')
C:\Users\Asimina\AppData\Local\Temp\ipykernel_25816\2450062250.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("Data/siss_geo_indicatori_lineari_2017.csv", sep=",", encoding='utf-8')
C:\Users\Asimina\AppData\Local\Temp\ipykernel_25816\2450062250.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("Data/siss_geo_indicatori_lineari_2018.csv", sep=",", encoding='utf-8')
C:\Users\Asimina\AppData\Local\Temp\ipykernel_25816\2450062250.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv("Data/siss_geo_indicatori_li

In [3]:
del df1, df2, df3, df4, df5, df6, df7
gc.collect()

20

Create unique geolocation

In [4]:
unprocessed = df.copy()

In [5]:
df['geometria'] = gpd.GeoSeries.from_wkt(df['geometria'])

In [6]:
pd.set_option('display.max_columns', 100)  # or however many you need
pd.set_option('display.float_format', '{:.4f}'.format)
gdf1 = gpd.GeoDataFrame(df, geometry='geometria')
gdf1['length'] = gdf1.geometry.length
gdf1['centroid'] = gdf1.geometry.centroid
# fix uuids
gdf1['uid'] = gdf1['uuid'].str.upper()
gdf1['uid'] = gdf1['uid'].astype(str)                         # convert all to string
gdf1['uid'] = gdf1['uid'].str.strip()                         # remove leading/trailing whitespace
gdf1['uid'] = gdf1['uid'].str.replace(r'\s+', ' ', regex=True)  # normalise spaces
gdf1['uid'] = gdf1['uid'].str.replace(r'[^\x00-\x7F]+', '', regex=True)  # remove non-ASCII
# find longest geometry for each uuid
helper = gdf1.sort_values(by=['uid','length'], ascending=[True, False])
helper['maxgeometry'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['geometria'].shift(), helper['geometria'])#
helper['maxlength'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['length'].shift(), helper['length'])  
helper_0 = helper.groupby('uid', as_index=False).first()
gdf1['maxgeometry'] = gdf1['uid'].map(helper_0.set_index('uid')['maxgeometry']).fillna(0)
gdf1['maxlength'] = gdf1['uid'].map(helper_0.set_index('uid')['maxlength']).fillna(0)
# find accident for each uuid and find also max length for each uuid
accidents = gdf1[gdf1['nr_incidenti'] > 0]
helper = accidents.sort_values(by=['uid','length'], ascending=[True, False])
helper['maxgeometry_accidents'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['geometria'].shift(), helper['geometria'])#
helper['maxlength_accidents'] = np.where(helper['uid'] == helper['uid'].shift()  , helper['length'].shift(), helper['length'])  
helper_0 = helper.groupby('uid', as_index=False).first()
gdf1['maxgeometry_accidents'] = gdf1['uid'].map(helper_0.set_index('uid')['maxgeometry_accidents']).fillna(0)
gdf1['maxlength_accidents'] = gdf1['uid'].map(helper_0.set_index('uid')['maxlength_accidents']).fillna(0)
# if accidents in multiple years, keep the one with the longest length
helper = accidents.groupby('uid', as_index=False, sort=False).agg({'geometria':'nunique','Year':'nunique','nr_incidenti':'sum'})
gdf1['accident_in_uuid'] = gdf1.uid.isin(helper.uid).astype(int)
acc_in_many_years_2 = helper[(helper['Year'] > 1) & (helper['geometria'] > 1)]
gdf1['acc_in_many_years'] = gdf1.uid.isin(acc_in_many_years_2.uid).astype(int)
gdf1['u_geometria'] = np.where(gdf1['accident_in_uuid'] == 1 , gdf1['maxgeometry_accidents'], gdf1['maxgeometry'])
gdf1['u_length'] = np.where(gdf1['accident_in_uuid'] == 1 , gdf1['maxlength_accidents'], gdf1['maxlength'])
# gdf1.head()

In [7]:
unprocessed['uid'] = unprocessed['uuid'].str.upper()

In [8]:
test = unprocessed[unprocessed['uid'] == '093CB267-6A29-2909-E054-0003BA0F36E6'] 
test.head(10)

,uuid,fk_l3,es_amm_nom,fk_l2,tp_str_nom,cod_comune,desc_comune,flusso_medio_gior,length_el_str,nr_incidenti,nr_incidenti_feriti,nr_incidenti_morti,nr_feriti,nr_morti,inc_tr,inc_morti_tr,inc_feriti_tr,t_incidentalita_flusso_tr,t_incidentalita_morti_flusso_tr,t_incidentalita_feriti_flusso_tr,t_mortalita_flusso_tr,t_lesivita_flusso_tr,t_ferimento_flusso_tr,group_strada,flusso_medio_gior_s,length_str_s,nr_incidenti_s,nr_incidenti_feriti_s,nr_incidenti_morti_s,nr_feriti_s,nr_morti_s,inc_s,inc_morti_s,inc_feriti_s,t_incidentalita_flusso_s,t_incidentalita_morti_flusso_s,t_incidentalita_feriti_flusso_s,t_mortalita_flusso_s,t_lesivita_flusso_s,t_ferimento_flusso_s,class_line1_t,class_symbol1_t,class_line1_s,class_line2_t,class_symbol2_t,class_line2_s,class_line3_t,class_symbol3_t,class_line3_s,class_line4_t,class_symbol4_t,class_line4_s,class_line5_t,class_symbol5_t,class_line5_s,class_line6_t,class_symbol6_t,class_line6_s,class_line7_t,class_symbol7_t,class_line7_s,class_line8_t,class_symbol8_t,class_line8_s,class_line9_t,class_symbol9_t,class_line9_s,geometria,Year,uid
349776,093CB267-6A29-2909-E054-0003BA0F36E6,0.0000,NaN,18189.0000,VIA ARSENALE,1272.0000,TORINO,NaN,157.1680,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,fkl2_18189,0,705.5760,3,3,0,3,0,3,0,3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406021361 4991490.99985228,...",2016,093CB267-6A29-2909-E054-0003BA0F36E6
102019,093CB267-6A29-2909-E054-0003BA0F36E6,0.0000,NaN,18189.0000,VIA ARSENALE,1272.0000,TORINO,0.0000,157.1680,1,1,0,1,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,fkl2_18189,0,705.5760,2,2,0,4,0,2,0,2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406021361 4991490.99985228,...",2017,093CB267-6A29-2909-E054-0003BA0F36E6
200471,093CB267-6A29-2909-E054-0003BA0F36E6,0.0000,NaN,18189.0000,VIA ARSENALE,1272.0000,TORINO,NaN,157.1680,1,1,0,1,0,1,0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,fkl2_18189,0,705.5760,3,3,0,6,0,3,0,3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406021361 4991490.99985228,...",2018,093CB267-6A29-2909-E054-0003BA0F36E6
300152,093cb267-6a29-2909-e054-0003ba0f36e6,0.0000,NaN,18189.0000,Via Arsenale,1272.0000,TORINO,0.0000,157.1680,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,fkl2_18189,0,705.5750,1,1,0,2,0,1,0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406 4991491, 396058.188 499...",2019,093CB267-6A29-2909-E054-0003BA0F36E6
217635,093cb267-6a29-2909-e054-0003ba0f36e6,NaN,NaN,18189.0000,VIA ARSENALE,NaN,TORINO,NaN,157.1680,2,2,0,3,0,2,0,2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,fkl2_18189,0,705.5750,3,3,0,7,0,3,0,3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406 4991491, 396058.188 499...",2020,093CB267-6A29-2909-E054-0003BA0F36E6
199861,093cb267-6a29-2909-e054-0003ba0f36e6,NaN,NaN,18189.0000,VIA ARSENALE,NaN,TORINO,NaN,157.1680,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,fkl2_18189,0,705.5750,5,5,0,6,0,5,0,5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"LINESTRING (396133.406 4991491, 396058.188 499...",2021,093CB267-6A29-2909-E054-0003BA0F36E6
494282,093cb267-6a29-2909-e054-0003ba0f36e6,NaN,NaN,18189.0000,VIA ARSENALE,1272.0000,TORINO,40.0000,157.1680,2,2,0,2,0,2,0,2,871.5916,0.0000,871.5916,0.0000,871.5916,871.5916,fkl2_18189,4876,705.5750,7,7,0,7,0,7,0,7,5.5744,0.0000,5.5744,0.0000,5.5744,5.5744,1,0,2,0,0,0,1,0,2,5,5,4,0,0,0,5,5,3,0,0,0,5,5,4,5,5,3,"LINESTRING (396133.406 4991491, 396058.188 499...",2022,093CB267-6A29-2909-E054-0003BA0F36E6


In [9]:
oacc = gdf1[(gdf1['accident_in_uuid'] == 1)]
oacc.to_csv("original_accidents_extras.csv", index=False)

In [10]:
del helper, helper_0, acc_in_many_years_2 #, gdf1, distances, possible_matches, possible_matches_index, buffer, bounds, idx, row, neighbour_indices, total_accidents 
gc.collect()

0

In [11]:
tranformed_df = gdf1.drop(columns=['maxgeometry', 'maxlength','maxgeometry_accidents','maxlength_accidents','acc_in_many_years'])

cols_to_drop = tranformed_df.columns[[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]]

# Drop those columns
tranformed_df = tranformed_df.drop(columns=cols_to_drop)

'Pivot'

In [12]:
#pivot on uids per year 
pivoted = tranformed_df.pivot_table(index='uid', columns='Year', values=['nr_incidenti','flusso_medio_gior','t_incidentalita_flusso_tr','length_el_str'], aggfunc='first')
pivoted.columns = [f"{col}_{year}" for col, year in pivoted.columns]
pivoted = pivoted.reset_index()
pivoted.head()

,uid,flusso_medio_gior_2016,flusso_medio_gior_2017,flusso_medio_gior_2018,flusso_medio_gior_2019,flusso_medio_gior_2020,flusso_medio_gior_2021,flusso_medio_gior_2022,length_el_str_2016,length_el_str_2017,length_el_str_2018,length_el_str_2019,length_el_str_2020,length_el_str_2021,length_el_str_2022,nr_incidenti_2016,nr_incidenti_2017,nr_incidenti_2018,nr_incidenti_2019,nr_incidenti_2020,nr_incidenti_2021,nr_incidenti_2022,t_incidentalita_flusso_tr_2016,t_incidentalita_flusso_tr_2017,t_incidentalita_flusso_tr_2018,t_incidentalita_flusso_tr_2019,t_incidentalita_flusso_tr_2020,t_incidentalita_flusso_tr_2021,t_incidentalita_flusso_tr_2022
0,00023EBC-4B49-4784-9286-C7A1A9758BE9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3730,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,0002ADD7-C661-4AC2-A06B-D0FC0ED7910D,NaN,NaN,NaN,NaN,987.0000,1043.0000,1087.0000,NaN,NaN,NaN,NaN,185.4950,185.4950,185.4950,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000
2,00047385-FEA9-4649-967A-145EA5595E66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.8350,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
3,0006C434-7195-4EF5-94B0-0C621DDF25C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.3310,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,000BCF81-5F7D-447D-8071-9368858ED27A,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,12.8610,12.8610,12.8610,12.8610,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000


In [13]:
cols_to_fix = [col for col in pivoted.columns if 'flusso_medio_gior' in col]
# Replace zeros with NaN in columns that flow is zero
pivoted[cols_to_fix] = pivoted[cols_to_fix].replace(0, np.nan) 
pivoted.head()

,uid,flusso_medio_gior_2016,flusso_medio_gior_2017,flusso_medio_gior_2018,flusso_medio_gior_2019,flusso_medio_gior_2020,flusso_medio_gior_2021,flusso_medio_gior_2022,length_el_str_2016,length_el_str_2017,length_el_str_2018,length_el_str_2019,length_el_str_2020,length_el_str_2021,length_el_str_2022,nr_incidenti_2016,nr_incidenti_2017,nr_incidenti_2018,nr_incidenti_2019,nr_incidenti_2020,nr_incidenti_2021,nr_incidenti_2022,t_incidentalita_flusso_tr_2016,t_incidentalita_flusso_tr_2017,t_incidentalita_flusso_tr_2018,t_incidentalita_flusso_tr_2019,t_incidentalita_flusso_tr_2020,t_incidentalita_flusso_tr_2021,t_incidentalita_flusso_tr_2022
0,00023EBC-4B49-4784-9286-C7A1A9758BE9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3730,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,0002ADD7-C661-4AC2-A06B-D0FC0ED7910D,NaN,NaN,NaN,NaN,987.0000,1043.0000,1087.0000,NaN,NaN,NaN,NaN,185.4950,185.4950,185.4950,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000
2,00047385-FEA9-4649-967A-145EA5595E66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.8350,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
3,0006C434-7195-4EF5-94B0-0C621DDF25C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.3310,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,000BCF81-5F7D-447D-8071-9368858ED27A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8610,12.8610,12.8610,12.8610,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000


In [14]:
#add columns that should not be pivoted
fixed_cols = tranformed_df[['uid', 'u_length','u_geometria','desc_comune']].drop_duplicates(subset='uid')
final_df = fixed_cols.merge(pivoted, on='uid', how='left')
final_df.head()

,uid,u_length,u_geometria,desc_comune,flusso_medio_gior_2016,flusso_medio_gior_2017,flusso_medio_gior_2018,flusso_medio_gior_2019,flusso_medio_gior_2020,flusso_medio_gior_2021,flusso_medio_gior_2022,length_el_str_2016,length_el_str_2017,length_el_str_2018,length_el_str_2019,length_el_str_2020,length_el_str_2021,length_el_str_2022,nr_incidenti_2016,nr_incidenti_2017,nr_incidenti_2018,nr_incidenti_2019,nr_incidenti_2020,nr_incidenti_2021,nr_incidenti_2022,t_incidentalita_flusso_tr_2016,t_incidentalita_flusso_tr_2017,t_incidentalita_flusso_tr_2018,t_incidentalita_flusso_tr_2019,t_incidentalita_flusso_tr_2020,t_incidentalita_flusso_tr_2021,t_incidentalita_flusso_tr_2022
0,0A400430-D303-662B-E054-0003BA0F36E6,63.3158,"LINESTRING (404099.56231678 4936222.49983281, ...",SALMOUR,705.0000,715.0000,655.0000,653.0000,5726.0000,NaN,802.0000,NaN,63.3160,63.3160,63.3150,63.3150,63.3150,63.3150,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
1,093CB25B-83AD-2909-E054-0003BA0F36E6,1142.1047,"LINESTRING (423570.125150913 4894530.99976198,...",GARESSIO,500.0000,500.0000,400.0000,400.0000,700.0000,915.0000,961.0000,NaN,1142.1050,1142.1050,1142.1050,1142.1050,1142.1050,1142.1050,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,5.9971,0.0000,0.0000,0.0000,0.0000
2,093CB25A-589D-2909-E054-0003BA0F36E6,276.7404,"LINESTRING (385798.625 5011371, 385854.187 501...",VILLANOVA CANAVESE,1467.0000,1437.0000,1457.0000,1459.0000,476.0000,486.0000,1584.0000,276.7400,276.7400,276.7400,276.7400,276.7400,276.7400,276.7400,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,6.7485,NaN,0.0000,0.0000,0.0000,0.0000,12.5000
3,0A400430-EC08-662B-E054-0003BA0F36E6,42.0672,"LINESTRING (401452.156 4930481, 401471.219 493...",TRINITA',3936.0000,4016.0000,3789.0000,3758.0000,3068.0000,3569.0000,5121.0000,NaN,42.0670,42.0670,42.0670,42.0670,42.0670,42.0670,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
4,093CB262-F73C-2909-E054-0003BA0F36E6,175.4540,"LINESTRING (459098.312576002 5059619.50013973,...",BORGOMANERO,13626.0000,13426.0000,13414.0000,13245.0000,8386.0000,8072.0000,7651.0000,175.4540,175.4540,175.4540,175.4540,175.4540,175.4540,175.4540,3.0000,0.0000,0.0000,2.0000,1.0000,0.0000,1.0000,3.4379,NaN,0.0000,2.3579,1.8620,0.0000,2.0409


In [16]:
import re

# Extract all years from the column names (integers)
years = sorted({
    int(re.search(r"_(\d{4})$", col).group(1))
    for col in final_df.columns 
    if col.startswith("flusso_medio_gior_") or col.startswith("nr_incidenti_")
})

import numpy as np

def compute_metrics(row):
    results = {}
    flusso_vals = []
    incidenti_vals = []
    
    for y in years:
        f_col = f"flusso_medio_gior_{y}"
        i_col = f"nr_incidenti_{y}"
        
        # Collect means independently (all non-NaN)
        if f_col in row and pd.notna(row[f_col]):
            flusso_vals.append(row[f_col])
        if i_col in row and pd.notna(row[i_col]):
            incidenti_vals.append(row[i_col])
        
        # Compute yearly incident rate only if both exist
        if f_col in row and i_col in row and pd.notna(row[f_col]) and pd.notna(row[i_col]):
            rate = (10**6) * row[i_col] / (365 * row[f_col] * row["u_length"] * 0.001)
            results[f"accident_flow_rate_{y}"] = rate
        else:
            results[f"accident_flow_rate_{y}"] = np.nan
    
    # Averages
    results["mean_accidents"] = np.mean(incidenti_vals) if incidenti_vals else np.nan
    results["mean_flow"] = np.mean(flusso_vals) if flusso_vals else np.nan
    # results["total_accidents"] = np.sum(incidenti_vals) if flusso_vals else np.nan
    
    return pd.Series(results)

# Apply row-wise
final_df = final_df.join(final_df.apply(compute_metrics, axis=1))


- length ---> some uids different geometria, unified them using the max (we dont have missing values) [to show some examples of uids with different geometrias]
- mean flow per year ---> this allows combining info from all years and giving the broad picture for the uuid instead of assessing it per year
- mean accidents ---> consider again the broad picture 

also this combo is more stabl since otherwise if 0 accident and then 2 in the net year, the rate for the first year would be 0, the other 2/length * flow (high deviation) while ours would be 1/length * flow (smoother)

Note: in daily flow, zeros are considered null and are excluded from the dataset

In [17]:
# # Select columns containing 'flusso_medio_gior'

accident_cols = [col for col in final_df.columns if col.startswith('nr_incidenti_')]
final_df['total_accidents'] = final_df[accident_cols].sum(axis=1, skipna=True)
# final_df["non_nan_acc"] = final_df[accident_cols].notna().sum(axis=1)
final_df["nan_acc"] = final_df[accident_cols].isna().sum(axis=1)

rate_cols = [col for col in final_df.columns if col.startswith('t_incidentalita_flusso_tr_')]
final_df['mean_incidentalita_flusso'] = final_df[rate_cols].mean(axis=1, skipna=True)
# final_df["non_nan_inc_flusso"] = final_df[rate_cols].notna().sum(axis=1)

flow_cols = [col for col in final_df.columns if col.startswith('flusso_medio_gior_') and not col.startswith('flusso_medio_gior_s_')]
# final_df["non_nan_flusso"] = final_df[flow_cols].notna().sum(axis=1)
final_df["nan_flusso"] = final_df[flow_cols].isna().sum(axis=1)

final_df['missing_inc_or_flu'] = final_df["nan_flusso"] + final_df["nan_acc"]
length_cols = [col for col in final_df.columns if col.startswith('length_el_str_')]
final_df["dif_length"] = (final_df[length_cols].max(axis=1) - final_df[length_cols].min(axis=1)) > 5
final_df["mean_length_el_str"] = final_df[length_cols].mean(axis=1, skipna=True)

In [18]:
final_df["dif_length_negligible"] = (final_df[length_cols].max(axis=1) - final_df[length_cols].min(axis=1)) > 0.5


In [19]:
print(final_df[final_df["dif_length_negligible"]== True].shape[0],final_df[final_df["dif_length_negligible"]== False].shape[0])

53053 486969


In [20]:
final_df['mean_accident_flow_rate'] = (10**6)*final_df['mean_accidents'] / (365*final_df['mean_flow'] * final_df['u_length']*0.001)

In [48]:
cols_to_front = ['uid','u_length','mean_accident_flow_rate','missing_inc_or_flu','mean_incidentalita_flusso','mean_accidents','total_accidents','mean_flow','mean_length_el_str','dif_length']

final_df = final_df[cols_to_front + [col for col in final_df.columns if col not in cols_to_front]]

final_df.head()

,uid,u_length,mean_accident_flow_rate,missing_inc_or_flu,mean_incidentalita_flusso,mean_accidents,total_accidents,mean_flow,mean_length_el_str,dif_length,u_geometria,desc_comune,flusso_medio_gior_2016,flusso_medio_gior_2017,flusso_medio_gior_2018,flusso_medio_gior_2019,flusso_medio_gior_2020,flusso_medio_gior_2021,flusso_medio_gior_2022,length_el_str_2016,length_el_str_2017,length_el_str_2018,length_el_str_2019,length_el_str_2020,length_el_str_2021,length_el_str_2022,nr_incidenti_2016,nr_incidenti_2017,nr_incidenti_2018,nr_incidenti_2019,nr_incidenti_2020,nr_incidenti_2021,nr_incidenti_2022,t_incidentalita_flusso_tr_2016,t_incidentalita_flusso_tr_2017,t_incidentalita_flusso_tr_2018,t_incidentalita_flusso_tr_2019,t_incidentalita_flusso_tr_2020,t_incidentalita_flusso_tr_2021,t_incidentalita_flusso_tr_2022,accident_flow_rate_2016,accident_flow_rate_2017,accident_flow_rate_2018,accident_flow_rate_2019,accident_flow_rate_2020,accident_flow_rate_2021,accident_flow_rate_2022,nan_acc,nan_flusso,dif_length_negligible
0,0A400430-D303-662B-E054-0003BA0F36E6,63.3158,0.0000,1,0.0000,0.0000,0.0000,1542.6667,63.3153,False,"LINESTRING (404099.56231678 4936222.49983281, ...",SALMOUR,705.0000,715.0000,655.0000,653.0000,5726.0000,NaN,802.0000,NaN,63.3160,63.3160,63.3150,63.3150,63.3150,63.3150,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0,1,False
1,093CB25B-83AD-2909-E054-0003BA0F36E6,1142.1047,0.5482,0,0.9995,0.1429,1.0000,625.1429,1142.1050,False,"LINESTRING (423570.125150913 4894530.99976198,...",GARESSIO,500.0000,500.0000,400.0000,400.0000,700.0000,915.0000,961.0000,NaN,1142.1050,1142.1050,1142.1050,1142.1050,1142.1050,1142.1050,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,5.9971,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,5.9971,0.0000,0.0000,0.0000,0.0000,0,0,False
2,093CB25A-589D-2909-E054-0003BA0F36E6,276.7404,3.5501,0,3.2081,0.4286,3.0000,1195.1429,276.7400,False,"LINESTRING (385798.625 5011371, 385854.187 501...",VILLANOVA CANAVESE,1467.0000,1437.0000,1457.0000,1459.0000,476.0000,486.0000,1584.0000,276.7400,276.7400,276.7400,276.7400,276.7400,276.7400,276.7400,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,6.7485,NaN,0.0000,0.0000,0.0000,0.0000,12.5000,6.7485,0.0000,0.0000,0.0000,0.0000,0.0000,12.5000,0,0,False
3,0A400430-EC08-662B-E054-0003BA0F36E6,42.0672,0.0000,0,0.0000,0.0000,0.0000,3893.8571,42.0670,False,"LINESTRING (401452.156 4930481, 401471.219 493...",TRINITA',3936.0000,4016.0000,3789.0000,3758.0000,3068.0000,3569.0000,5121.0000,NaN,42.0670,42.0670,42.0670,42.0670,42.0670,42.0670,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,False
4,093CB262-F73C-2909-E054-0003BA0F36E6,175.4540,1.4046,0,1.6165,1.0000,7.0000,11117.1429,175.4540,False,"LINESTRING (459098.312576002 5059619.50013973,...",BORGOMANERO,13626.0000,13426.0000,13414.0000,13245.0000,8386.0000,8072.0000,7651.0000,175.4540,175.4540,175.4540,175.4540,175.4540,175.4540,175.4540,3.0000,0.0000,0.0000,2.0000,1.0000,0.0000,1.0000,3.4379,NaN,0.0000,2.3579,1.8620,0.0000,2.0409,3.4379,0.0000,0.0000,2.3579,1.8620,0.0000,2.0409,0,0,False


below is accidents

In [72]:
acc = final_df[(final_df['total_accidents'] > 0)]
acc = gpd.GeoDataFrame(acc, geometry='u_geometria')

In [73]:
# Sort dataframe by mean_accidents in descending order
acc = acc.sort_values(by='mean_accidents', ascending=False)
acc['mean_accident_div_length'] = acc['mean_accidents'] / (acc['u_length']*0.001)#(10**6)*final_df['mean_accidents'] / (final_df['u_length']*0.001)

# Count how many years have non-NaN values in nr_incidenti_YEAR columns
accident_cols = [col for col in acc.columns if col.startswith('nr_incidenti_')]
acc['years_with_data'] = acc[accident_cols].notna().sum(axis=1)

# Extract starting and ending years with non-NaN values
import re

def get_year_range(row):
    years_with_values = []
    for col in accident_cols:
        # Extract year from column name (e.g., 'nr_incidenti_2016' -> 2016)
        match = re.search(r'_(\d{4})$', col)
        if match and pd.notna(row[col]):
            years_with_values.append(int(match.group(1)))
    
    if years_with_values:
        return pd.Series({'year_start': min(years_with_values), 'year_end': max(years_with_values)})
    else:
        return pd.Series({'year_start': np.nan, 'year_end': np.nan})

acc[['year_start', 'year_end']] = acc.apply(get_year_range, axis=1)

In [74]:
# Calculate trend of accidents over all years and over recent years
from sklearn.linear_model import LinearRegression

def calculate_trends(row):
    # Collect all years with data and their accident values
    year_accident_pairs = []
    for col in accident_cols:
        match = re.search(r'_(\d{4})$', col)
        if match and pd.notna(row[col]):
            year = int(match.group(1))
            accidents = row[col]
            year_accident_pairs.append((year, accidents))
    
    if len(year_accident_pairs) < 3:
        return pd.Series({'trend_all_years': np.nan})
    
    # Sort by year
    year_accident_pairs.sort(key=lambda x: x[0])
    years = np.array([x[0] for x in year_accident_pairs])
    accidents = np.array([x[1] for x in year_accident_pairs])
    
    # Overall trend: linear regression slope across all years
    X_all = years.reshape(-1, 1)
    y_all = accidents
    model_all = LinearRegression()
    model_all.fit(X_all, y_all)
    trend_all = model_all.coef_[0]
        
    return pd.Series({'trend_all_years': trend_all})

acc[['trend_all_years']] = acc.apply(calculate_trends, axis=1)

In [75]:
# Compute early average (first 4 years before the last 3), last-3 average, last-3 trend,
# and scaled versions of key metrics
import re
from sklearn.preprocessing import MinMaxScaler

# Helper: get sorted (year, value) pairs for accidents for a row
accident_year_pattern = re.compile(r"_(\d{4})$")

def get_year_value_pairs(row):
    pairs = []
    for col in accident_cols:
        m = accident_year_pattern.search(col)
        if m:
            y = int(m.group(1))
            v = row[col]
            if pd.notna(v):
                pairs.append((y, float(v)))
    pairs.sort(key=lambda x: x[0])
    return pairs

# Row-wise compute early average, last-3 average, and last-3 trend
from sklearn.linear_model import LinearRegression

def compute_window_metrics(row):
    pairs = get_year_value_pairs(row)
    if len(pairs) == 0:
        return pd.Series({
            'avg_early': np.nan,
            'avg_last_3': np.nan,
            'trend_last_3_years': np.nan
        })

    years = [p[0] for p in pairs]
    values = [p[1] for p in pairs]

    # Average of the first 4 years BEFORE the last 3 years with data
    if len(years) > 3:
        pre_years = years[:-3]  # everything before the last 3
        pre_values = values[:-3]
        if len(pre_years) >= 1:
            # take first 4 from the pre block
            take_n = min(4, len(pre_values))
            avg_early = float(np.mean(pre_values[:take_n]))
        else:
            avg_early = np.nan
    else:
        avg_early = np.nan

    # Average of last up to 3 years
    take_last = min(3, len(values))
    avg_last_3 = float(np.mean(values[-take_last:])) if take_last >= 1 else np.nan

    # Trend of last up to 3 years (need at least 2 points)
    if take_last >= 2:
        X = np.array(years[-take_last:]).reshape(-1, 1)
        y = np.array(values[-take_last:])
        lr = LinearRegression()
        lr.fit(X, y)
        trend_last_3 = float(lr.coef_[0])
    else:
        trend_last_3 = np.nan

    return pd.Series({
        'avg_early': avg_early,
        'avg_last_3': avg_last_3,
        'trend_last_3_years': trend_last_3
    })

metrics = acc.apply(compute_window_metrics, axis=1)
acc[['avg_early','avg_last_3','trend_last_3_years']] = metrics[['avg_early','avg_last_3','trend_last_3_years']]

# Prepare columns to scale (0-1). If constant or all-NaN, scaled to 0.
cols_to_scale = [
    'avg_early',              # new
    'mean_accidents',         # average of all years (existing)
    'avg_last_3',             # new
    'total_accidents',        # existing
    'trend_all_years',        # existing
    'trend_last_3_years'      # new
]

for c in cols_to_scale:
    if c in acc.columns:
        s = acc[c].astype(float)
        # handle all-NaN or constant series
        if s.notna().sum() == 0 or (s.max(skipna=True) == s.min(skipna=True)):
            acc[f'{c}_scaled'] = s.apply(lambda x: np.nan if pd.isna(x) else 0.0)
        else:
            col_min = s.min(skipna=True)
            col_max = s.max(skipna=True)
            acc[f'{c}_scaled'] = (s - col_min) / (col_max - col_min)


In [85]:
acc['combined_metric'] = 0.1*acc['avg_early_scaled'] + 0.45*acc['avg_last_3_scaled'] + 0.3*acc['trend_last_3_years_scaled'] + 0.15*acc['trend_all_years_scaled']
# Bring new metrics and their scaled versions to the front (after existing front columns)
# Existing front columns (as set previously) - scaled columns will be inserted after trend_recent_years
base_front = ['uid','combined_metric','mean_accidents','total_accidents','trend_all_years','years_with_data','avg_early','avg_last_3','trend_last_3_years','avg_early_scaled','mean_accidents_scaled','avg_last_3_scaled','total_accidents_scaled','trend_last_3_years_scaled','trend_all_years_scaled','mean_accident_flow_rate','mean_accident_div_length','year_start','year_end']

other_cols = [c for c in acc.columns if c not in base_front]
acc = acc[base_front + other_cols]

In [86]:
# Reorder columns to put nr_incidenti_YEAR columns right after u_length before saving
all_cols = list(acc.columns)
accident_cols_list = sorted([c for c in all_cols if c.startswith('nr_incidenti_')], key=lambda x: int(re.search(r'_(\d{4})$', x).group(1)))
other_cols_list = [c for c in all_cols if not c.startswith('nr_incidenti_')]
if 'u_length' in other_cols_list:
    u_idx = other_cols_list.index('u_length')
    reordered_cols = other_cols_list[:u_idx+1] + accident_cols_list + other_cols_list[u_idx+1:]
    acc = acc[reordered_cols]
acc.head(1)

,uid,combined_metric,mean_accidents,total_accidents,trend_all_years,years_with_data,avg_early,avg_last_3,trend_last_3_years,avg_early_scaled,mean_accidents_scaled,avg_last_3_scaled,total_accidents_scaled,trend_last_3_years_scaled,trend_all_years_scaled,mean_accident_flow_rate,mean_accident_div_length,year_start,year_end,u_length,nr_incidenti_2016,nr_incidenti_2017,nr_incidenti_2018,nr_incidenti_2019,nr_incidenti_2020,nr_incidenti_2021,nr_incidenti_2022,missing_inc_or_flu,mean_incidentalita_flusso,mean_flow,mean_length_el_str,dif_length,u_geometria,desc_comune,flusso_medio_gior_2016,flusso_medio_gior_2017,flusso_medio_gior_2018,flusso_medio_gior_2019,flusso_medio_gior_2020,flusso_medio_gior_2021,flusso_medio_gior_2022,length_el_str_2016,length_el_str_2017,length_el_str_2018,length_el_str_2019,length_el_str_2020,length_el_str_2021,length_el_str_2022,t_incidentalita_flusso_tr_2016,t_incidentalita_flusso_tr_2017,t_incidentalita_flusso_tr_2018,t_incidentalita_flusso_tr_2019,t_incidentalita_flusso_tr_2020,t_incidentalita_flusso_tr_2021,t_incidentalita_flusso_tr_2022,accident_flow_rate_2016,accident_flow_rate_2017,accident_flow_rate_2018,accident_flow_rate_2019,accident_flow_rate_2020,accident_flow_rate_2021,accident_flow_rate_2022,nan_acc,nan_flusso,dif_length_negligible
156242,093CB26A-0595-2909-E054-0003BA0F36E6,0.7447,13.7143,96.0000,-0.6786,7,15.7500,11.0000,5.0000,1.0000,1.0000,0.8250,1.0000,0.7600,0.3032,3.1344,18.2160,2016,2022,752.8683,11.0000,20.0000,22.0000,10.0000,6.0000,11.0000,16.0000,0,3.1239,15922.2857,752.8680,False,"LINESTRING (385473.375 4992982, 385531.5 49929...",RIVOLI,18148.0000,17037.0000,16074.0000,17308.0000,13631.0000,14485.0000,14773.0000,NaN,752.8680,752.8680,752.8680,752.8680,752.8680,752.8680,2.2057,4.2719,4.9807,2.1025,1.6018,2.7635,3.9413,2.2057,4.2719,4.9807,2.1025,1.6018,2.7635,3.9413,0,0,False


In [ ]:
# cols_to_scale = ['combined_metric']
# for c in cols_to_scale:
#     if c in acc.columns:
#         s = acc[c].astype(float)
#         # handle all-NaN or constant series
#         if s.notna().sum() == 0 or (s.max(skipna=True) == s.min(skipna=True)):
#             acc[f'{c}_scaled'] = s.apply(lambda x: np.nan if pd.isna(x) else 0.0)
#         else:
#             col_min = s.min(skipna=True)
#             col_max = s.max(skipna=True)
#             acc[f'{c}_scaled'] = (s - col_min) / (col_max - col_min)


In [87]:
acc.to_csv('data.csv', index=False)